In [1]:
from dataloader import Data
import torchvision.transforms as transforms
import constants as C
from utils import split_labels
import numpy as np

mode = 'test'

file_root = '../splits/Shopping100k'
img_root_path = '/Users/simone/Desktop/VMR/Dataset/Shopping100k/Images'

if mode == 'train':
        data = Data(file_root,  img_root_path, 
                          transforms.Compose([
                              transforms.Resize((C.TRAIN_INIT_IMAGE_SIZE, C.TRAIN_INIT_IMAGE_SIZE)),
                              transforms.RandomHorizontalFlip(),
                              transforms.CenterCrop(C.TARGET_IMAGE_SIZE),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=C.IMAGE_MEAN, std=C.IMAGE_STD)
                          ]), 'train')
elif mode == 'test':
    data = Data(file_root,  img_root_path,
                  transforms.Compose([
                      transforms.Resize((C.TARGET_IMAGE_SIZE, C.TARGET_IMAGE_SIZE)),
                      transforms.ToTensor(),
                      transforms.Normalize(mean=C.IMAGE_MEAN, std=C.IMAGE_STD)
                  ]), 'test')

attr_num = data.attr_num
labels = data.label_data

splitted_labels = [split_labels(lbl, attr_num) for lbl in labels]

In [7]:
from pprint import pprint
from tqdm import tqdm
import h5py
import numpy as np
from f_utils import create_n_manip

N = 8

def checkmanip(q_lbl, t_lbl, manip_list):

    actual = q_lbl
    for manip in manip_list:
       actual = np.add(actual, manip)
       actual_splitted = split_labels(actual, attr_num)
       for act in actual_splitted:
            if np.count_nonzero((act == 1) | (act == 0)) != act.size:
                print("there are numbers different from 0 or 1!!")
                print(act)
                return False
            if np.count_nonzero(act == 1) > 1:
                print("there are more than one 1!!")
                print(act)
                return False
    #check if result is t
    if not np.array_equal(actual, t_lbl):
        print("target is not the same!!")
        return False
    
    return True

hf = h5py.File(f'/home/falhamdoosh/disentagledFeaturesExtractor/multi_manip/{mode}/couples_N_{N}_small.h5', 'r')

for i, (q_id, t_id) in enumerate(zip(hf['q'], hf['t'])):
    q_lbl = labels[q_id]
    t_lbl = labels[t_id]
    #print(q_id, t_id)
    manips, scratch = create_n_manip(N, q_lbl, t_lbl)

    if not checkmanip(q_lbl, t_lbl, manips):
        print(q_id, t_id)
        assert 1 == 2 
    if i % 1000 == 0:
        print(f"done {i} out of {len(hf['q'])} ")

print('done')

done 0 out of 100000 
done 1000 out of 100000 
done 2000 out of 100000 
done 3000 out of 100000 
done 4000 out of 100000 
done 5000 out of 100000 
done 6000 out of 100000 
done 7000 out of 100000 
done 8000 out of 100000 
done 9000 out of 100000 
done 10000 out of 100000 
done 11000 out of 100000 
done 12000 out of 100000 
done 13000 out of 100000 
done 14000 out of 100000 
done 15000 out of 100000 
done 16000 out of 100000 
done 17000 out of 100000 
done 18000 out of 100000 
done 19000 out of 100000 
done 20000 out of 100000 
done 21000 out of 100000 
done 22000 out of 100000 
done 23000 out of 100000 
done 24000 out of 100000 
done 25000 out of 100000 
done 26000 out of 100000 
done 27000 out of 100000 
done 28000 out of 100000 
done 29000 out of 100000 
done 30000 out of 100000 
done 31000 out of 100000 
done 32000 out of 100000 
done 33000 out of 100000 
done 34000 out of 100000 
done 35000 out of 100000 
done 36000 out of 100000 
done 37000 out of 100000 
done 38000 out of 100000 

In [6]:
from f_utils import create_n_manip, listify_manip
import random
import importlib
import f_utils

importlib.reload(f_utils)

t_lbl = labels[105]
q_lbl = labels[100]

multi_manip =  np.subtract(t_lbl, q_lbl)
manip_list = listify_manip(multi_manip, 10)

manips, v = create_n_manip(10, t_lbl, q_lbl)
for i, m in enumerate(manips):
    m[0] = i
for m in manips:
    print(m)
np.random.shuffle(manips)
print("++++")
for m in manips:
    print(m)

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0]
[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0]
[ 2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  

In [12]:
import random

random.randint(0, 2)

1